Revoir minimum WR distances : très long 

In [ ]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from utils import simulate_tournament, simulate_n_tournaments, compare_settings, initiate_league


New LDC format places of interest :
* The 8 first teams qualify directly to round of 16
* From 9 to 24 : pseudo round of 32
* Interesting teams : i (9th) and y(25th)  

Format :
* 36 teams,
* 8 games,
* Hat determination of the first 2 games

LoL :
* 16 teams,
* Top 8 to Quarter finales,
* 6 games (not exactly, teams with 3 win qualifies, team with 3 loss gets eliminated)


In [55]:
simulate_tournament(6,3, delta_level = 'exponential',verbose_pairing = True, verbose_game=False)


round_number=1 - Successfull allocation based on ranking
round_number=2 - Successfull allocation based on ranking
round_number=3 - Successfull allocation based on ranking


,Level,Strategy,Nb_win,Nb_games,Win_rate,R1_opponent,R1_result,R2_opponent,R2_result,R3_opponent,R3_result
b,0.5,[],3,3,1.0,a,Win,f,Win,d,Win
d,0.125,[],2,3,0.666667,f,Win,c,Win,b,Loss
a,1.0,[],2,3,0.666667,b,Loss,e,Win,c,Win
c,0.25,[],1,3,0.333333,e,Win,d,Loss,a,Loss
e,0.0625,[],1,3,0.333333,c,Loss,a,Loss,f,Win
f,0.03125,[],0,3,0.0,d,Loss,b,Loss,e,Loss


In [52]:
compare_settings(n_tournaments=10, 
                 n_teams=6, 
                 n_rounds = 3, 
                 delta_level ='linear',
                 strategies={'d':[1]}, 
                 probabilistic=True, 
                 deterministic=True)


Probabilistic resolution


,Level,Strategy,Strategic_avg_WR,Control_avg_WR,Delta
a,1.0,[],0.766667,0.633333,0.133333
c,0.666667,[],0.733333,0.633333,0.1
b,0.833333,[],0.633333,0.666667,-0.033333
e,0.333333,[],0.466667,0.566667,-0.1
d,0.5,[1],0.233333,0.433333,-0.2
f,0.166667,[],0.166667,0.066667,0.1


---------------------------------------------------
Deterministic resolution


,Level,Strategy,Strategic_avg_WR,Control_avg_WR,Delta
a,1.0,[],1.0,1.0,0.0
b,0.833333,[],0.666667,0.666667,0.0
c,0.666667,[],0.6,0.633333,-0.033333
d,0.5,[1],0.4,0.433333,-0.033333
e,0.333333,[],0.266667,0.266667,0.0
f,0.166667,[],0.066667,0.0,0.066667


# Matching algorithm sandbox

In [ ]:
from utils import assign_opponents, play_round
n_teams = 16
n_games = 6


In [ ]:
possible_games_matrix, league_table = initiate_league(n_teams, n_games) 
for i in range(n_games-1):
    possible_games_matrix, league_table = assign_opponents(possible_games_matrix,league_table,round_number=i+1, verbose = True)
    league_table = play_round(league_table,round_number=i+1)


In [ ]:
league_table
